I do not understand at all how bash works. But merging three answers from stack overflow gave me this.  
This lines of "code" used to download celebritites photos from celebA dataset and description of bounding boxes.  
This lines must be working independent of user, because zip folders are shared as "public"

In [ ]:
from google.colab import output

In [ ]:
!python -m pip install -U matplotlib

In [ ]:
!apt-get install jq
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7Zty7TyOcXVrig' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_1.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/EJnNERFOymOJ9Q' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_2.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/be-pzbsSyMUvgQ' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_3.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/eL8c-HCgSgXBOA' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_4.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/YHELGFERKfjb0Q' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_5.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/Tm9pGDZzZHMQAA' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_6.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/XaJKOGI7CMVRJw' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_7.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/dT9nSQ1FODIsew' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_8.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/zXLkF77fO1DDKw' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_9.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/-UKk194UkZQ4lA' | jq -r '.href' | xargs -I{} wget {} -O celeba_in_wild_10.zip
!curl -s 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/REzDeHs9o0wpjQ' | jq -r '.href' | xargs -I{} wget {} -O bboxes.txt
output.clear()

In [ ]:
!unzip ./celeba_in_wild_1.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_1.zip
!unzip ./celeba_in_wild_2.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_2.zip
!unzip ./celeba_in_wild_3.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_3.zip
!unzip ./celeba_in_wild_4.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_4.zip
!unzip ./celeba_in_wild_5.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_5.zip
!unzip ./celeba_in_wild_6.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_6.zip
!unzip ./celeba_in_wild_7.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_7.zip
!unzip ./celeba_in_wild_8.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_8.zip
!unzip ./celeba_in_wild_9.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_9.zip
!unzip ./celeba_in_wild_10.zip -d ./celebA_in_wild
#!rm -f ./celeba_in_wild_10.zip
!cp ./bboxes.txt ./celebA_in_wild
output.clear()

In [ ]:
!python -m pip install -U matplotlib
!git clone https://github.com/timesler/facenet-pytorch.git
!mv facenet-pytorch/models ./
!ls models

output.clear()

In [ ]:
!cp -r ./drive/MyDrive/detectionA_lib ./detectionA_lib

output.clear()

In [ ]:
import sys
sys.path.insert(0, './detectionA_lib')

In [ ]:
from detectionA_lib.dataset_preparation import splitter

splitter.train_val_test_split("./celebA_in_wild", "bboxes.txt")

In [ ]:
from detectionA_lib.detector import pyramidal_yolo
from detectionA_lib.dataset_preparation import celebA_detector_dataset
import torch
from torch.utils.data import DataLoader
from torchsummary import summary

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pyramidDetector = pyramidal_yolo.ResNetPyramidDetector().to(device)
min_square_side = 256

train_dataset = celebA_detector_dataset.wildCelebDataset("celebrities_splitted/train", "celebrities_splitted/train/bbox.txt", min_square_side)
val_dataset = celebA_detector_dataset.wildCelebDataset("celebrities_splitted/val", "celebrities_splitted/val/bbox.txt", min_square_side)
test_dataset = celebA_detector_dataset.wildCelebDataset("celebrities_splitted/test", "celebrities_splitted/test/bbox.txt", min_square_side)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True, drop_last = True)
val_dataloader = DataLoader(val_dataset, batch_size = 16, drop_last = True)
test_dataloader = DataLoader(test_dataset, batch_size = 16, drop_last = True)

In [14]:
optim = torch.optim.Adam(params = pyramidDetector.parameters(), lr = 1e-05)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, gamma = 0.1, milestones = [10000000])

### Померяем среднюю метрику IoU на test dataset:

In [18]:
import pickle
from detectionA_lib.metrics.IoU_metric import measure_meanIoU

with open("./drive/MyDrive/lib_Detector_Adam_Step_1e-04_10ep->1e-05_5ep/PyramidYoloDetector_epoch_=_1", "rb") as m_file:
    model = pickle.load(m_file).to(device)

print("Средняя метрика IoU на test dataset составляет: " + str(measure_meanIoU(test_dataloader, model).item()))

/content/detectionA_lib/detector/pyramidal_yolo.py:63: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  loc_best_tensor_y_coord = loc_best_tensor_idx_flattened//width


Средняя метрика IoU на test dataset составляет: 0.8150802850723267
